In [11]:
import geopandas as gpd
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import datetime
import os
import ipaddress
from aimlsse_api.client import GroundDataClient, SatelliteDataClient
from shapely import Polygon

In [2]:
ground_data_client = GroundDataClient(ipaddress.ip_address('127.0.0.1'), 8000)

In [4]:
stations_meta = ground_data_client.queryMetadata(polygons=[
    Polygon([(6.0, 47.5),(15.0, 47.5),(15.0, 55.0),(6.0, 55.0)])
])
print(stations_meta)

                      geometry    id                 name         plot_name  \
0     POINT (6.04222 54.03667)  EHHW          Buitengaats       Buitengaats   
1     POINT (6.58472 53.12361)  EHGG            Groningen  GRONINGEN/EELDE    
2    POINT (14.13806 47.53250)  LOXA  AIGEN/ENNSTAL (MIL)  AIGEN/ENNSTAL (M   
3    POINT (14.18751 48.23322)  LOWL                 Linz  LINZ (CIV/MIL)     
4    POINT (13.00000 47.80000)  LOWS             Salzburg  SALZBURG AIRPORT   
..                         ...   ...                  ...               ...   
108   POINT (7.51028 47.61417)  LFSB             Mulhouse  BALE/MULHOUSE      
109   POINT (6.22167 48.68778)  LFSN                Nancy  NANCY/ESSEY        
110   POINT (7.64028 48.54944)  LFST           Strasbourg  STRASBOURG/ENTZH   
111   POINT (6.21152 49.62658)  ELLX            Luxemburg  LUXEMBOURG (AUT)   
112  POINT (14.62278 53.39528)  EPSC             Szczecin  SZCZECIN/GOLENIO   

      network   latitude  longitude  elevation  ind

In [5]:
locations = stations_meta[['id', 'geometry']]
print(locations)

       id                   geometry
0    EHHW   POINT (6.04222 54.03667)
1    EHGG   POINT (6.58472 53.12361)
2    LOXA  POINT (14.13806 47.53250)
3    LOWL  POINT (14.18751 48.23322)
4    LOWS  POINT (13.00000 47.80000)
..    ...                        ...
108  LFSB   POINT (7.51028 47.61417)
109  LFSN   POINT (6.22167 48.68778)
110  LFST   POINT (7.64028 48.54944)
111  ELLX   POINT (6.21152 49.62658)
112  EPSC  POINT (14.62278 53.39528)

[113 rows x 2 columns]


In [6]:
satellite_data_client = SatelliteDataClient(ipaddress.ip_address('127.0.0.1'), 8010)

In [7]:
grid_cells = satellite_data_client.queryContainingGeometry(locations)
print(grid_cells)

                                              geometry cell_name  \
0    GEOMETRYCOLLECTION Z (POLYGON Z ((5.67154 47.8...     31TGN   
1    GEOMETRYCOLLECTION Z (POLYGON Z ((5.71882 48.7...     31UGP   
2    GEOMETRYCOLLECTION Z (POLYGON Z ((6.32793 47.8...     32TLT   
3    GEOMETRYCOLLECTION Z (POLYGON Z ((6.28064 48.7...     32ULU   
4    GEOMETRYCOLLECTION Z (POLYGON Z ((4.56477 55.0...     31UFA   
..                                                 ...       ...   
163  GEOMETRYCOLLECTION Z (POLYGON Z ((13.58809 50....     33UVR   
164  GEOMETRYCOLLECTION Z (POLYGON Z ((13.56055 51....     33UVS   
165  GEOMETRYCOLLECTION Z (POLYGON Z ((13.56055 51....     33UVS   
166  GEOMETRYCOLLECTION Z (POLYGON Z ((13.46869 54....     33UVV   
167  GEOMETRYCOLLECTION Z (POLYGON Z ((13.46869 54....     33UVV   

                                           Description  index_right    id  
0    TILE PROPERTIES<br><table border=0 cellpadding...          105  LFSX  
1    TILE PROPERTIES<br><table 

In [8]:
groups = grid_cells.groupby(['cell_name'])

In [12]:
target_group: gpd.GeoDataFrame = groups.get_group('32UMA')
target_stations: pd.Series = target_group['id']
print(target_stations)

62    ETOR
64    EDFE
65    EDDF
66    ETOU
Name: id, dtype: object


In [14]:
target_locations = locations[locations['id'].isin(target_stations)]
print(target_locations)

      id                  geometry
24  ETOR  POINT (8.46339 49.56357)
31  EDFE  POINT (8.64150 49.96083)
36  EDDF  POINT (8.59861 50.04639)
97  ETOU  POINT (8.32540 50.04982)


In [25]:
target_locations['geometry'].apply(lambda point: print(f'Point({point.x}, {point.y})'))

Point(8.46339, 49.56357)
Point(8.6415, 49.96083)
Point(8.59861, 50.04639)
Point(8.3254, 50.04982)


24    None
31    None
36    None
97    None
Name: geometry, dtype: geometry